In [1]:
import os
import json

import numpy as np
import pandas as pd
from google.colab import drive
#from google.cloud import firestore
#import firebase_admin
from firebase_admin import firestore#, credentials

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#JSON_KEY_FILE = '/content/drive/My Drive/firestore-446822-ac05de749f4a.json'

#JSON_KEY_FILE = "/content/drive/My Drive/service-pa-79ac5-0f0babe13506.json"

JSON_KEY_FILE = "/content/drive/My Drive/csg3101-service-providing-app-firebase-adminsdk-7xpfw-f39bc603db.json"

In [4]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = JSON_KEY_FILE

# Initialize Firebase Admin SDK
#cred = credentials.Certificate("/content/drive/My Drive/service-pa-79ac5-0f0babe13506.json")
#firebase_admin.initialize_app(cred)

In [5]:
JSON_DATA_FILE = '/content/drive/My Drive/yelp_academic_dataset_user.json'

In [6]:
# Define the number of lines to read
n_lines = 100

# Read the specified number of lines into a list of dictionaries
with open(JSON_DATA_FILE, "r") as file:
    data = [json.loads(next(file)) for _ in range(n_lines)]

# Read the JSON lines file directly into a pandas DataFrame
#df = pd.read_json(JSON_DATA_FILE, lines=True)

# Convert the list of dictionaries into a DataFrame
df = pd.DataFrame(data)

# Display the first few rows
print(df.head())

                  user_id    name  review_count        yelping_since  useful  \
0  qVc8ODYU5SZjKXVBgXdI7w  Walker           585  2007-01-25 16:47:26    7217   
1  j14WgRoU_-2ZE1aw1dXrJg  Daniel          4333  2009-01-25 04:35:42   43091   
2  2WnXYQFK0hXEoTxPtV2zvg   Steph           665  2008-07-25 10:41:00    2086   
3  SZDeASXq7o05mMNLshsdIA    Gwen           224  2005-11-29 04:38:33     512   
4  hA5lMy-EnncsH4JoR-hFGQ   Karen            79  2007-01-05 19:40:59      29   

   funny   cool                                              elite  \
0   1259   5994                                               2007   
1  13066  27281  2009,2010,2011,2012,2013,2014,2015,2016,2017,2...   
2   1010   1003                           2009,2010,2011,2012,2013   
3    330    299                                     2009,2010,2011   
4     15      7                                                      

                                             friends  fans  ...  \
0  NSCy54eWehBJyZdG2iE84w, pe42

In [7]:
print(len(df)) # total number of entries

100


In [8]:
# Select specific columns
df = df[['user_id', 'name']]

# Display the result
print(df.head())

                  user_id    name
0  qVc8ODYU5SZjKXVBgXdI7w  Walker
1  j14WgRoU_-2ZE1aw1dXrJg  Daniel
2  2WnXYQFK0hXEoTxPtV2zvg   Steph
3  SZDeASXq7o05mMNLshsdIA    Gwen
4  hA5lMy-EnncsH4JoR-hFGQ   Karen


In [9]:
# Rename columns
df = df.rename(columns={'user_id': 'customerId', 'name': 'cardHolderName'})

# Display the result
print(df.head())

               customerId cardHolderName
0  qVc8ODYU5SZjKXVBgXdI7w         Walker
1  j14WgRoU_-2ZE1aw1dXrJg         Daniel
2  2WnXYQFK0hXEoTxPtV2zvg          Steph
3  SZDeASXq7o05mMNLshsdIA           Gwen
4  hA5lMy-EnncsH4JoR-hFGQ          Karen


In [10]:
df['cardType'] = np.random.choice(['credit', 'debit'], size=len(df))

# Display the result
print(df.head())

               customerId cardHolderName cardType
0  qVc8ODYU5SZjKXVBgXdI7w         Walker    debit
1  j14WgRoU_-2ZE1aw1dXrJg         Daniel   credit
2  2WnXYQFK0hXEoTxPtV2zvg          Steph    debit
3  SZDeASXq7o05mMNLshsdIA           Gwen   credit
4  hA5lMy-EnncsH4JoR-hFGQ          Karen   credit


In [11]:
def generate_card_number(card_type="credit"):
    """
    Generate a random credit or debit card number using NumPy.

    :param card_type: 'credit' or 'debit'
    :return: A valid card number as a string.
    """
    # Define card number prefixes (e.g., Visa starts with 4, Mastercard starts with 51-55)
    prefixes = {
        "credit": ["4", "51", "52", "53", "54", "55"],
        "debit": ["60", "62", "65"]
    }

    if card_type not in prefixes:
        raise ValueError("Invalid card type. Choose 'credit' or 'debit'.")

    # Choose a random prefix
    prefix = np.random.choice(prefixes[card_type])
    length = 16  # Standard card number length

    # Generate the remaining digits except the last one (checksum)
    card_number = [int(d) for d in prefix]
    while len(card_number) < (length - 1):
        card_number.append(np.random.randint(0, 10))

    # Calculate the Luhn checksum
    def calculate_checksum(card_number):
        digits = card_number[::-1]  # Reverse the digits
        checksum = 0
        for i, digit in enumerate(digits):
            if i % 2 == 0:  # Double every second digit
                digit *= 2
                if digit > 9:
                    digit -= 9
            checksum += digit
        return checksum % 10

    # Add the checksum digit to make the card number valid
    checksum_digit = (10 - calculate_checksum(card_number)) % 10
    card_number.append(checksum_digit)

    return ''.join(map(str, card_number))

In [12]:
# Generate random card numbers
#random_credit_card = generate_card_number("credit")
#random_debit_card = generate_card_number("debit")

#print("Random Credit Card:", random_credit_card)
#print("Random Debit Card:", random_debit_card)

In [13]:
df['cardNumber'] = df['cardType'].apply(lambda x: generate_card_number(x))

In [14]:
# Display the result
print(df.head())

               customerId cardHolderName cardType        cardNumber
0  qVc8ODYU5SZjKXVBgXdI7w         Walker    debit  6224529086710014
1  j14WgRoU_-2ZE1aw1dXrJg         Daniel   credit  5293286816365258
2  2WnXYQFK0hXEoTxPtV2zvg          Steph    debit  6061421849036539
3  SZDeASXq7o05mMNLshsdIA           Gwen   credit  5574887190781349
4  hA5lMy-EnncsH4JoR-hFGQ          Karen   credit  5263527706104869


In [15]:
# Convert DataFrame to JSON
processed_data = df.to_dict(orient='records')

In [16]:
print(processed_data[0])

{'customerId': 'qVc8ODYU5SZjKXVBgXdI7w', 'cardHolderName': 'Walker', 'cardType': 'debit', 'cardNumber': '6224529086710014'}


In [17]:
print(len(processed_data))

100


In [18]:
# Initialize Firestore client
db = firestore.Client()

In [19]:
# Firestore collection name
collection_name = "customerCardDetails"

In [20]:
# Upload processed data to Firestore
for doc in processed_data:

  # Reference to your Firestore collection
  collection_ref = db.collection(collection_name)

  # Add the document with an auto-generated ID
  doc_ref = collection_ref.add(doc)

In [21]:
print("Processed data uploaded successfully!")

Processed data uploaded successfully!
